In [1]:
##NEW NOTEBOOK####


In [2]:
import pandas as pd
import numpy as np
import redis

In [3]:
host_name='redis-17239.c85.us-east-1-2.ec2.cloud.redislabs.com'
port=17239
pwd='ZCojZ6VRsSIqMd3GPu5lJRDKAF1PALNS'

In [4]:
r=redis.StrictRedis(host=host_name,port=port,password=pwd)

In [5]:
r.ping()

True

In [5]:
# r.ping was to check if the connection is correctly made or not

In [6]:
np_file=np.load('data_frame_compress2.npz',allow_pickle=True)

In [7]:
np_file.items()
x_values=np_file['arr_0']
col_names=np_file['arr_1']

In [8]:
df=pd.DataFrame(x_values,columns=col_names)

In [9]:
df

,name,role,features
0,ANGELINA JOLIE,STUDENT,"[0.1988192, 1.4748838, -0.097958855, -0.880505..."
1,BARACK OBAMA,TEACHER,"[-0.22177823, 1.5762169, -0.503078, 0.13601282..."
2,CHRIS EVANS,STUDENT,"[0.35691053, -0.41491055, 0.03921842, -0.15436..."
3,MORGAN FREEMAN,TEACHER,"[0.76278925, 0.89061487, 0.46873555, 0.2589808..."
4,SCARLETT JOHANSSON,STUDENT,"[-0.13535997, -0.2821385, -0.42700845, -0.2825..."


In [10]:
df['name_role']=df['name']+'@'+df['role']
df

,name,role,features,name_role
0,ANGELINA JOLIE,STUDENT,"[0.1988192, 1.4748838, -0.097958855, -0.880505...",ANGELINA JOLIE@STUDENT
1,BARACK OBAMA,TEACHER,"[-0.22177823, 1.5762169, -0.503078, 0.13601282...",BARACK OBAMA@TEACHER
2,CHRIS EVANS,STUDENT,"[0.35691053, -0.41491055, 0.03921842, -0.15436...",CHRIS EVANS@STUDENT
3,MORGAN FREEMAN,TEACHER,"[0.76278925, 0.89061487, 0.46873555, 0.2589808...",MORGAN FREEMAN@TEACHER
4,SCARLETT JOHANSSON,STUDENT,"[-0.13535997, -0.2821385, -0.42700845, -0.2825...",SCARLETT JOHANSSON@STUDENT


In [12]:
records=df[['name_role','features']].to_dict(orient='records')

In [13]:
len(records)

5

In [14]:
for record in records:
        name_role=record['name_role']
        vector=record['features']
        vector_bytes=vector.tobytes()
        r.hset(name='academy:reister2',key=name_role,value=vector_bytes)

In [15]:
name='academy:reister2'
retrive_dict=r.hgetall(name)
retrive_series=pd.Series(retrive_dict)
retrive_series=retrive_series.apply(lambda x:np.frombuffer(x,dtype=np.float32))
index=retrive_series.index
index= list(map(lambda x:x.decode(),index))
retrive_series.index=index
retrive_df=retrive_series.to_frame().reset_index()
retrive_df.columns=['name_role','features']


In [16]:
retrive_df

,name_role,features
0,BARACK OBAMA@TEACHER,"[-0.22177823, 1.5762169, -0.503078, 0.13601282..."
1,ANGELINA JOLIE@STUDENT,"[0.1988192, 1.4748838, -0.097958855, -0.880505..."
2,CHRIS EVANS@STUDENT,"[0.35691053, -0.41491055, 0.03921842, -0.15436..."
3,SCARLETT JOHANSSON@STUDENT,"[-0.13535997, -0.2821385, -0.42700845, -0.2825..."
4,MORGAN FREEMAN@TEACHER,"[0.76278925, 0.89061487, 0.46873555, 0.2589808..."


In [17]:
retrive_df

,name_role,features
0,BARACK OBAMA@TEACHER,"[-0.22177823, 1.5762169, -0.503078, 0.13601282..."
1,ANGELINA JOLIE@STUDENT,"[0.1988192, 1.4748838, -0.097958855, -0.880505..."
2,CHRIS EVANS@STUDENT,"[0.35691053, -0.41491055, 0.03921842, -0.15436..."
3,SCARLETT JOHANSSON@STUDENT,"[-0.13535997, -0.2821385, -0.42700845, -0.2825..."
4,MORGAN FREEMAN@TEACHER,"[0.76278925, 0.89061487, 0.46873555, 0.2589808..."


In [18]:
import cv2
import insightface
import pandas as pd
import numpy as np
import redis

In [19]:
from insightface.app import FaceAnalysis

In [20]:
Face_app=FaceAnalysis(name='buffalo_sc',root='insight_model',provides=['CUDAExecutionProvider','CPUExecutionProvider'])

C:\Users\Jay satish patel\Desktop\p_j\attendance_system\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insight_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insight_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5


In [21]:
Face_app.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)

set det-size: (640, 640)


In [22]:
for i in range(3):
    person_name=input("enter the name of person")
    role=input('''please choose if you are 
    1.student
    2.teacher
    enter number 1 or 2''')
    if(role in('1','2')):
        if(role=='1'):
            role='STUDENT'
        elif(role=='2'):
            role='TEACHER'
        break
        
    else:
            print("invalid input,Try again")
       
            



enter the name of person JAYPATE
please choose if you are 
    1.student
    2.teacher
    enter number 1 or 2 1


In [23]:
key=person_name+'@'+role


In [24]:
import cv2
 # Import the FaceApp module (if it's a custom module)

cap = cv2.VideoCapture(0)
face_embedding = []
sample = 0

# Create an instance of FaceApp (assuming it's a custom class)


while True:
    ret, frame = cap.read()

    if not ret:
        print('Camera not opened')
        break

    # Assuming FaceApp.get() returns a list of face detections
    result = Face_app.get(frame, max_num=1)
    
    for res in result:
        x1, y1, x2, y2 = res['bbox']  # Extract bounding box coordinates
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convert to integers
        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
        embed = res['embedding']
        face_embedding.append(embed)
        sample = sample + 1

    if sample >= 200:
        break

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [25]:
len(face_embedding)

200

In [26]:
x_mean=np.asarray(face_embedding).mean(axis=0)
x_mean.shape

(512,)

In [27]:
x_mean_bytes=x_mean.tobytes()

In [30]:
r.hset(name='academy:reister2', key=key, value=x_mean_bytes)


1

In [33]:
retrive_df

,name_role,features
0,BARACK OBAMA@TEACHER,"[-0.22177823, 1.5762169, -0.503078, 0.13601282..."
1,ANGELINA JOLIE@STUDENT,"[0.1988192, 1.4748838, -0.097958855, -0.880505..."
2,CHRIS EVANS@STUDENT,"[0.35691053, -0.41491055, 0.03921842, -0.15436..."
3,SCARLETT JOHANSSON@STUDENT,"[-0.13535997, -0.2821385, -0.42700845, -0.2825..."
4,MORGAN FREEMAN@TEACHER,"[0.76278925, 0.89061487, 0.46873555, 0.2589808..."
